<a href="https://colab.research.google.com/github/fabio-weydson/sandbox-datascience/blob/main/CE_5_1_por_genes_autismo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo de caso 5.1 - Identificando novos genes do autismo

Configuração do *notebook*:

Sincronize sua conta do Google. Para isso, siga o link que aparece na saída da seguinte célula uma vez executada. Copie o código que aparece na tela e insira-o na saída da célula. Assim que visualizar a mensagem: `Google Drive sincronizado com sucesso!`poderá continuar executando o restante das células.

In [ ]:
from google.colab import auth
auth.authenticate_user()

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
data_drop = drive.CreateFile({'id':'1Iin0h_JR8qiJl8UmJfa7cq-YZ4ou5n2Q'})
data_drop.GetContentFile('BIOGRID.txt')
data_drop = drive.CreateFile({'id':'10FxN4lVk8-dsbdmVLEWEwoKI1wSFAqKs'})
data_drop.GetContentFile('gene-id-table.txt')
data_drop = drive.CreateFile({'id':'1yWjhKnw2knYtLeeX5mzFrJ-qrtEQgAOx'})
data_drop.GetContentFile('gene-score.csv')
data_drop = drive.CreateFile({'id':'1P4MPEtbrQuIUHraBy3nxCKieGFHjK4HG'})
data_drop.GetContentFile('steiner_tree.R')

print('Google Drive sincronizado com sucesso!')

In [ ]:
%load_ext rpy2.ipython

Instalar e importar bibliotecas: (ignorar resultados a não ser que não apareça a frase: `Bibliotecas instaladas com sucesso!`)

In [ ]:
%%R
install.packages('igraph')
cat('\nBibliotecas instaladas com sucesso!')

In [ ]:
%%R
library(igraph) 
cat('\nBibliotecas instaladas com sucesso!')

## Dados


### Rede de interações proteína-proteína

Carregar o banco de dados BIOGRID.

In [ ]:
%%R
biogrid <- read.delim("BIOGRID.txt",stringsAsFactors = F)

Ver os diferentes componentes do banco de dados.

In [ ]:
%%R
names(biogrid)  

Anexar o banco de dados ao *workspace* e atribuir as interações entre proteínas à variável `HSnet`.

In [ ]:
%%R
attach(biogrid) 
HSnet <- graph.data.frame(data.frame(Entrez.Gene.Interactor.A,Entrez.Gene.Interactor.B),directed=F)

Também podemos ver a forma que essa rede de interações adquire:

In [ ]:
%%R
graph.data.frame(data.frame(Entrez.Gene.Interactor.A,Entrez.Gene.Interactor.B),directed=F)

A linha seguinte permitirá que você trace a rede, mas isso levará algum tempo. O resultado será de melhor qualidade se você executar a tarefa em um ambiente nativo de R (como o RStudio).

In [ ]:
%%R
plot(HSnet)

Agora podemos obter a matriz de adjacência da rede, $A$. Como a matriz é muito grande, podemos visualizar a primeira seção de 15x15 células:


In [ ]:
%%R
A <- get.adjacency(HSnet)
A[1:15,1:15] 

In [ ]:
%%R
#o seguinte é FALSE se o grafo não for simples.
is.simple(HSnet)

Como observamos que existem arestas múltiplas na rede, vamos transformá-la em uma rede simples:

In [ ]:
%%R
#eliminar arestas múltiplas e autoloops
HSnet <- simplify(HSnet, remove.multiple = TRUE, remove.loops = TRUE, edge.attr.comb = getIgraphOpt("edge.attr.comb"))

is.simple(HSnet)

Observamos que agora só há arestas únicas:

In [ ]:
%%R
A <- get.adjacency(HSnet)
A[1:15,1:15]

In [ ]:
%%R
plot(HSnet)

Para esta aplicação, são removidos os nós de alto grau; esses nós normalmente representam genes constituintes que são necessários para manter uma célula viva, mas geralmente não são específicos para uma doença em particular.

In [ ]:
%%R
overly.attached.proteins <- which(degree(HSnet)>1000)
overly.attached.proteins

In [ ]:
%%R
HSnet <- delete.vertices(HSnet, overly.attached.proteins)

#o seguinte é TRUE se o grafo estiver conectado.
is.connected(HSnet)

### Genes causadores do autismo

Agora, passamos a ler a tabela `gene-id`:

In [ ]:
%%R
gene.table <- read.delim("gene-id-table.txt") 
names(gene.table)

Depois, temos que ler também a tabela de pontuação para genes relacionados ao autismo (`gene-scores`):

In [ ]:
%%R
gene.score<-read.csv("gene-score.csv",stringsAsFactors=F) 
attach(gene.score)
names(gene.score)

In [ ]:
%%R 
#exibindo as pontuações
unique(Score)

Agora podemos identificar esses genes com pontuações significativas:

In [ ]:
%%R
signif.scores <- c("1S","1","2S","2","3S","3") 
signif.genes <- Gene.Symbol[which(Score %in% signif.scores)] 
signif.EIDs <- gene.table[which(gene.table[,1] %in% signif.genes),2]

Finalmente, podemos usar a rede de interação `HSnet` que criamos anteriormente, para determinar os genes que estão presentes na rede e que são conhecidos por causar autismo:

In [ ]:
%%R
geneEIDs <- as.numeric(V(HSnet)$name)
HSnetN <- HSnet
V(HSnetN)$name <- 1:length(V(HSnet))
signif.ids <- which(geneEIDs %in% signif.EIDs) 
length(signif.ids)

## Desenvolvendo um interactoma do autismo

Importa-se o script para calcular o `steiner_tree.R`, ou seja, a sub-rede de menor comprimento que conecta certo conjunto de nós.

In [ ]:
%%R
source('steiner_tree.R')

Identificar o *Steiner Tree* e cronometrar o tempo que leva para executar essa função.

In [ ]:
%%R
system.time(HS.stree <- steiner_tree(terminals=signif.ids, graph=HSnetN))

Mostrar a sobreposição entre a significância do autismo e vértices *Steiner Tree*

In [ ]:
%%R
length(intersect(signif.ids,V(HS.stree)$name)) 
labels<-gene.table[as.numeric(V(HS.stree)$name),1] 
labels<-as.character(labels)

Identificar os genes que têm pontuações significativas e atribuir a eles a cor vermelha

In [ ]:
%%R
colors<-rep("skyblue",length(V(HS.stree))) 
colors[which(as.numeric(V(HS.stree)$name) %in% signif.ids)] = "red"
#Atribuir cor aos vértices da árvore
V(HS.stree)$color = colors

Exportar o novo gráfico para um arquivo `.pdf`. Você pode acessá-lo na barra lateral.

In [ ]:
%%R
pdf("ASD_interactome.pdf",width=12, height=12) 
system.time(plot(HS.stree,vertex.label=labels,vertex.size=5,vertex.label.cex=0.8)) 
dev.off()

## Análise e propriedades do interactoma do autismo

Para analisar as propriedades do interactoma do autismo e compará-las com os interactomas gerados aleatoriamente, usaremos a biblioteca `sna`:

In [ ]:
%%R
install.packages('sna')
library(sna, quietly=TRUE)

Primeiro, criamos uma função que calcula as pontuações de conectividade da rede. Neste estudo, as pontuações são os diâmetros da rede e a distância geodésica média entre dois nós quaisquer:

In [ ]:
%%R
c.scores<-function(graph){
    n <- length(V(graph))
    sp <- shortest.paths(graph)
    neighbors <- sum(sp==1)/2
    neighbors2 <- sum(sp==2)/2 
    return(c(2*neighbors/(n*(n-1)),2*neighbors2/(n*(n-1))))
}

clus <- clusters(HSnetN,mode=c("weak")) 
connected.ids <- which(clus$membership==1) 
length(connected.ids)

Geramos N sub-redes aleatoriamente. Isso pode levar algum tempo se N for grande.

In [ ]:
%%R
N<-50 
strees<-list(N) 
effs<-numeric(N) 
nei<-numeric(N) 
nei2<-numeric(N) 
for (i in 1:N){
    new.ids<-sample(x=connected.ids,size=length(signif.ids)) 
    strees[[i]] <- steiner_tree(terminals=new.ids,graph=HSnetN) 
    effs[i]<-efficiency(get.adjacency(strees[[i]],sparse=F)) 
    cs<-c.scores(strees[[i]])
    nei[i]<-cs[1]
    nei2[i]<-cs[2] 
}

Podemos mostrar as pontuações de eficiência e conectividade para cada um dos grafos aleatórios:

In [ ]:
%%R
effs

In [ ]:
%%R 
nei 

In [ ]:
%%R 
nei2

Finalmente, mostramos as pontuações de eficiência e conectividade para o interactoma do autismo.


In [ ]:
%%R
efficiency(get.adjacency(HS.stree,sparse=F)) 

In [ ]:
%%R
c.scores(HS.stree)

## Identificação de novos genes candidatos

Calcular a pontuação de centralidade de intermediação para cada nó

In [ ]:
%%R
betweeness_centrality_scores = igraph::betweenness(HS.stree)

Identificar os que que ainda não sabemos se são significativos


In [ ]:
%%R
significant_centrality = c()
count = 0
for (i in 1:length(betweeness_centrality_scores)){
    if (!(as.numeric(names(betweeness_centrality_scores[i])) %in% signif.ids)) { 
        significant_centrality = c(significant_centrality,betweeness_centrality_scores[i])
    }
}

Ordenar e mostrar o número de nós significativos

In [ ]:
%%R
significant_centrality = sort(significant_centrality, decreasing=TRUE) 
length(significant_centrality)